In [318]:
import boto3 #The AWS SDK for Python (Boto3) provides a Python API for AWS infrastructure services.
import pandas as pd
from io import StringIO, BytesIO

from datetime import datetime
from datetime import timedelta

## Get data from the bucket and load them into df

In [319]:
arg_date = '2022-01-01'
src_bucket='xetra-1234'
trg_bucket='my-xetra-cz'

#filter columns I need
columns=['ISIN', 'Date', 'Time', 'StartPrice', 'MaxPrice', 'MinPrice','EndPrice', 'TradedVolume']

#name for the output file
key = 'xetra_daily_report_' + datetime.today().strftime("%Y%m%d_%H%M%S") + '.parquet'

#define how many days you want to download
days_download=10

In [320]:
#day before
arg_date_dt = pd.to_datetime(arg_date) - timedelta(days=1)
#arg_date_dt = datetime.strptime(arg_date, '%Y-%m-%d').date() - timedelta(days=1)

arg_date_dt

Timestamp('2021-12-31 00:00:00')

In [321]:
s3 = boto3.resource('s3')
bucket = s3.Bucket(src_bucket)

#some files don't have data - StockMarket is closed at weekends - this one has '2022-01-28/2022-01-28_BINS_XETR16.csv'
#split key of the file by a slash '2022-01-27/2022-01-27_BINS_XETR00.csv' and select the first part:2022-01-27
objects = [obj for obj in bucket.objects.all() if pd.to_datetime(obj.key.split("/")[0]) >= arg_date_dt and pd.to_datetime(obj.key.split("/")[0]) <= arg_date_dt+timedelta(days=days_download)]


In [322]:
objects[0:5]

[s3.ObjectSummary(bucket_name='xetra-1234', key='2022-01-03/2022-01-03_BINS_XETR00.csv'),
 s3.ObjectSummary(bucket_name='xetra-1234', key='2022-01-03/2022-01-03_BINS_XETR01.csv'),
 s3.ObjectSummary(bucket_name='xetra-1234', key='2022-01-03/2022-01-03_BINS_XETR02.csv'),
 s3.ObjectSummary(bucket_name='xetra-1234', key='2022-01-03/2022-01-03_BINS_XETR03.csv'),
 s3.ObjectSummary(bucket_name='xetra-1234', key='2022-01-03/2022-01-03_BINS_XETR04.csv')]

In [323]:
len(objects)

144

In [324]:
""" #get columns names from the first file=dataframe cannot be empty
csv_obj_init = bucket.Object(key=objects[0].key).get().get('Body').read().decode('utf-8') #[0] = first file in the list
#read into a memory
data = StringIO(csv_obj_init) #read into a memory
#read columns
df_init = pd.read_csv(data, delimiter=',') 

#display the columns
df_init.columns """

" #get columns names from the first file=dataframe cannot be empty\ncsv_obj_init = bucket.Object(key=objects[0].key).get().get('Body').read().decode('utf-8') #[0] = first file in the list\n#read into a memory\ndata = StringIO(csv_obj_init) #read into a memory\n#read columns\ndf_init = pd.read_csv(data, delimiter=',') \n\n#display the columns\ndf_init.columns "

In [325]:
#define function for concating the df
def csv_to_df(filename):
    csv_obj = bucket.Object(key=filename).get().get('Body').read().decode('utf-8')
    data = StringIO(csv_obj)
    df = pd.read_csv(data, delimiter=',')
    return df

#call the function
df_all = pd.concat([csv_to_df(obj.key) for obj in objects if not csv_to_df(obj.key).empty ], ignore_index=True)

In [326]:
df_all

,ISIN,Mnemonic,SecurityDesc,SecurityType,Currency,SecurityID,Date,Time,StartPrice,MaxPrice,MinPrice,EndPrice,TradedVolume,NumberOfTrades
0,AT0000A0E9W5,SANT,S+T AG O.N.,Common stock,EUR,2504159,2022-01-03,08:00,14.7600,14.7600,14.7500,14.7500,4414,2
1,DE000A0DJ6J9,S92,SMA SOLAR TECHNOL.AG,Common stock,EUR,2504287,2022-01-03,08:00,37.6400,37.6600,37.6000,37.6600,1649,3
2,DE000A0D6554,NDX1,NORDEX SE O.N.,Common stock,EUR,2504290,2022-01-03,08:00,13.9900,14.0300,13.9400,13.9600,23011,36
3,DE000A0D9PT0,MTX,MTU AERO ENGINES NA O.N.,Common stock,EUR,2504297,2022-01-03,08:00,180.0000,180.0500,179.5000,179.5000,2308,22
4,DE000A0HN5C6,DWNI,DEUTSCHE WOHNEN SE INH,Common stock,EUR,2504314,2022-01-03,08:00,37.2800,37.2800,37.2800,37.2800,2897,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
708225,DE000A2NB650,MUX,MUTARES KGAA NA O.N.,Common stock,EUR,3425967,2022-01-10,16:43,22.9000,22.9000,22.9000,22.9000,126,2
708226,DE000BASF111,BAS,BASF SE NA O.N.,Common stock,EUR,2504663,2022-01-10,16:44,66.0900,66.0900,66.0900,66.0900,200,2
708227,DE0007030009,RHM,RHEINMETALL AG,Common stock,EUR,2505071,2022-01-10,16:44,86.9800,86.9800,86.9800,86.9800,260,3
708228,IE00BLRPRH06,VIXL,WITR MU.AS.I.VIX ETP 2062,ETN,EUR,5902384,2022-01-10,16:47,2.3318,2.3318,2.3318,2.3318,122,2


In [327]:
#crop columns
df_all=df_all.loc[:,columns]
df_all

,ISIN,Date,Time,StartPrice,MaxPrice,MinPrice,EndPrice,TradedVolume
0,AT0000A0E9W5,2022-01-03,08:00,14.7600,14.7600,14.7500,14.7500,4414
1,DE000A0DJ6J9,2022-01-03,08:00,37.6400,37.6600,37.6000,37.6600,1649
2,DE000A0D6554,2022-01-03,08:00,13.9900,14.0300,13.9400,13.9600,23011
3,DE000A0D9PT0,2022-01-03,08:00,180.0000,180.0500,179.5000,179.5000,2308
4,DE000A0HN5C6,2022-01-03,08:00,37.2800,37.2800,37.2800,37.2800,2897
...,...,...,...,...,...,...,...,...
708225,DE000A2NB650,2022-01-10,16:43,22.9000,22.9000,22.9000,22.9000,126
708226,DE000BASF111,2022-01-10,16:44,66.0900,66.0900,66.0900,66.0900,200
708227,DE0007030009,2022-01-10,16:44,86.9800,86.9800,86.9800,86.9800,260
708228,IE00BLRPRH06,2022-01-10,16:47,2.3318,2.3318,2.3318,2.3318,122


In [328]:
df_all.shape

(708230, 8)

In [329]:
df_all.dropna(inplace=True)
df_all.shape

(708230, 8)

## Get opening and closing price

In [330]:
#get the first value of the 'StartPrice' column within each group
df_all['opening_price'] = df_all.sort_values(by=['Time' ]).groupby(['ISIN', 'Date'])['StartPrice'].transform('first')
df_all.head(10)

,ISIN,Date,Time,StartPrice,MaxPrice,MinPrice,EndPrice,TradedVolume,opening_price
0,AT0000A0E9W5,2022-01-03,08:00,14.76,14.760,14.75,14.75,4414,14.76
1,DE000A0DJ6J9,2022-01-03,08:00,37.64,37.660,37.60,37.66,1649,37.64
2,DE000A0D6554,2022-01-03,08:00,13.99,14.030,13.94,13.96,23011,13.99
3,DE000A0D9PT0,2022-01-03,08:00,180.00,180.050,179.50,179.50,2308,180.00
4,DE000A0HN5C6,2022-01-03,08:00,37.28,37.280,37.28,37.28,2897,37.28
5,DE000A0JL9W6,2022-01-03,08:00,60.00,60.000,59.35,60.00,8883,60.00
6,DE000A0LD2U1,2022-01-03,08:00,19.53,19.530,19.53,19.53,16703,19.53
7,DE000A0LD6E6,2022-01-03,08:00,85.30,85.800,85.20,85.80,1726,85.30
8,DE000A0WMPJ6,2022-01-03,08:00,18.17,18.175,18.06,18.06,13598,18.17
9,DE000A0Z2ZZ5,2022-01-03,08:00,23.40,23.400,23.39,23.39,4237,23.40


In [331]:
#check the opening price is set correctly in 'opening price' column
df_all.query('ISIN == "AT0000A0E9W5"').head()

,ISIN,Date,Time,StartPrice,MaxPrice,MinPrice,EndPrice,TradedVolume,opening_price
0,AT0000A0E9W5,2022-01-03,08:00,14.76,14.76,14.75,14.75,4414,14.76
157,AT0000A0E9W5,2022-01-03,08:01,14.80,14.83,14.80,14.83,2214,14.76
266,AT0000A0E9W5,2022-01-03,08:02,14.85,14.87,14.85,14.85,1634,14.76
572,AT0000A0E9W5,2022-01-03,08:03,14.85,14.85,14.85,14.85,1110,14.76
3045,AT0000A0E9W5,2022-01-03,08:05,14.82,14.82,14.80,14.80,5690,14.76


In [332]:
#same with closing price
df_all['closing_price'] = df_all.sort_values(by=['Time']).groupby(['ISIN', 'Date'])['StartPrice'].transform('last')

In [333]:
#check the opening price is set correctly in 'opening price' column
df_all.query('ISIN == "AT0000A0E9W5"').tail()

,ISIN,Date,Time,StartPrice,MaxPrice,MinPrice,EndPrice,TradedVolume,opening_price,closing_price
702090,AT0000A0E9W5,2022-01-10,16:22,14.11,14.13,14.11,14.13,3078,14.6,14.12
702750,AT0000A0E9W5,2022-01-10,16:24,14.14,14.14,14.10,14.10,5466,14.6,14.12
704044,AT0000A0E9W5,2022-01-10,16:28,14.11,14.11,14.11,14.11,121,14.6,14.12
704433,AT0000A0E9W5,2022-01-10,16:29,14.11,14.15,14.10,14.15,1834,14.6,14.12
705025,AT0000A0E9W5,2022-01-10,16:35,14.12,14.12,14.12,14.12,26193,14.6,14.12


## Aggregations

In [334]:
#add columns by calculations
df_all = (df_all.groupby(['ISIN', 'Date'], as_index=False)
          .agg(opening_price_eur=('opening_price', 'min'), 
               closing_price_eur=('closing_price', 'min'), 
               minimum_price_eur=('MinPrice', 'min'), 
               maximum_price_eur=('MaxPrice', 'max'), 
               daily_traded_volume=('TradedVolume', 'sum')
               )
            )
df_all

,ISIN,Date,opening_price_eur,closing_price_eur,minimum_price_eur,maximum_price_eur,daily_traded_volume
0,AT000000STR1,2022-01-03,36.550,37.400,36.350,37.400,660
1,AT000000STR1,2022-01-04,37.750,37.850,37.750,37.850,27
2,AT000000STR1,2022-01-05,37.700,37.200,37.200,37.700,800
3,AT000000STR1,2022-01-06,36.600,37.300,36.600,37.300,140
4,AT000000STR1,2022-01-07,36.850,37.350,36.850,37.350,475
...,...,...,...,...,...,...,...
19142,XS2376095068,2022-01-04,41.186,41.360,41.186,41.360,0
19143,XS2376095068,2022-01-05,40.990,41.024,40.772,41.024,250
19144,XS2376095068,2022-01-06,38.154,37.854,37.854,38.154,7
19145,XS2376095068,2022-01-07,36.862,36.710,36.710,37.412,2541


In [335]:
#check the opening price is set correctly in 'opening price' column
df_all.query('ISIN == "AT0000A0E9W5"').head()

,ISIN,Date,opening_price_eur,closing_price_eur,minimum_price_eur,maximum_price_eur,daily_traded_volume
153,AT0000A0E9W5,2022-01-03,14.76,15.62,14.75,15.70,370776
154,AT0000A0E9W5,2022-01-04,15.66,14.96,14.96,15.68,589461
155,AT0000A0E9W5,2022-01-05,14.96,14.65,14.50,15.16,568340
156,AT0000A0E9W5,2022-01-06,14.41,14.71,14.30,14.75,397740
157,AT0000A0E9W5,2022-01-07,14.75,14.47,14.39,14.91,155276


## Percent change from previous closing

In [336]:
df_all['prev_closing_price'] = df_all.sort_values(by=['Date']).groupby(['ISIN'])['closing_price_eur'].shift(1)
df_all.head(10)

,ISIN,Date,opening_price_eur,closing_price_eur,minimum_price_eur,maximum_price_eur,daily_traded_volume,prev_closing_price
0,AT000000STR1,2022-01-03,36.55,37.40,36.35,37.40,660,NaN
1,AT000000STR1,2022-01-04,37.75,37.85,37.75,37.85,27,37.40
2,AT000000STR1,2022-01-05,37.70,37.20,37.20,37.70,800,37.85
3,AT000000STR1,2022-01-06,36.60,37.30,36.60,37.30,140,37.20
4,AT000000STR1,2022-01-07,36.85,37.35,36.85,37.35,475,37.30
5,AT000000STR1,2022-01-10,37.40,36.60,36.50,37.40,64,37.35
6,AT00000FACC2,2022-01-03,7.39,7.65,7.39,7.67,1045,NaN
7,AT00000FACC2,2022-01-04,7.86,7.79,7.79,7.95,681,7.65
8,AT00000FACC2,2022-01-05,7.93,7.79,7.60,7.93,532,7.79
9,AT00000FACC2,2022-01-06,7.61,7.59,7.59,7.61,707,7.79


In [337]:
df_all['change_prev_closing_%'] = (df_all['closing_price_eur'] - df_all['prev_closing_price']) / df_all['prev_closing_price'] * 100
df_all.head(5)

,ISIN,Date,opening_price_eur,closing_price_eur,minimum_price_eur,maximum_price_eur,daily_traded_volume,prev_closing_price,change_prev_closing_%
0,AT000000STR1,2022-01-03,36.55,37.40,36.35,37.40,660,NaN,NaN
1,AT000000STR1,2022-01-04,37.75,37.85,37.75,37.85,27,37.40,1.203209
2,AT000000STR1,2022-01-05,37.70,37.20,37.20,37.70,800,37.85,-1.717305
3,AT000000STR1,2022-01-06,36.60,37.30,36.60,37.30,140,37.20,0.268817
4,AT000000STR1,2022-01-07,36.85,37.35,36.85,37.35,475,37.30,0.134048


In [338]:
df_all.drop(columns=['prev_closing_price'], inplace=True)

df_all = df_all.round(decimals=2)

df_all.head(5)

,ISIN,Date,opening_price_eur,closing_price_eur,minimum_price_eur,maximum_price_eur,daily_traded_volume,change_prev_closing_%
0,AT000000STR1,2022-01-03,36.55,37.40,36.35,37.40,660,NaN
1,AT000000STR1,2022-01-04,37.75,37.85,37.75,37.85,27,1.20
2,AT000000STR1,2022-01-05,37.70,37.20,37.20,37.70,800,-1.72
3,AT000000STR1,2022-01-06,36.60,37.30,36.60,37.30,140,0.27
4,AT000000STR1,2022-01-07,36.85,37.35,36.85,37.35,475,0.13


In [339]:
df_all.shape

(19147, 8)

In [340]:
#filter irrelevatn dates
df_all = df_all.query('Date >= @arg_date')

In [341]:
df_all.shape

(19147, 8)

## Write to your S3 bucket

In [342]:
out_buffer = BytesIO()
df_all.to_parquet(out_buffer, index=False)
bucket_target = s3.Bucket(trg_bucket)
bucket_target.put_object(Body=out_buffer.getvalue(), Key=key)

s3.Object(bucket_name='my-xetra-cz', key='xetra_daily_report_20231216_201047.parquet')

## Reading the uploaded file

In [343]:
for obj in bucket_target.objects.all():
    print(obj.key)

xetra_daily_report_20231216_200620.parquet
xetra_daily_report_20231216_201047.parquet


In [344]:
prq_obj = bucket_target.Object(key='xetra_daily_report_20231216_200620.parquet').get().get('Body').read()
data = BytesIO(prq_obj)
df_report = pd.read_parquet(data)

In [345]:
df_report

,ISIN,Date,opening_price_eur,closing_price_eur,minimum_price_eur,maximum_price_eur,daily_traded_volume,change_prev_closing_%
0,AT000000STR1,2022-01-03,36.55,37.40,36.35,37.40,660,NaN
1,AT000000STR1,2022-01-04,37.75,37.85,37.75,37.85,27,1.20
2,AT000000STR1,2022-01-05,37.70,37.20,37.20,37.70,800,-1.72
3,AT000000STR1,2022-01-06,36.60,37.30,36.60,37.30,140,0.27
4,AT000000STR1,2022-01-07,36.85,37.35,36.85,37.35,475,0.13
...,...,...,...,...,...,...,...,...
19142,XS2376095068,2022-01-04,41.19,41.36,41.19,41.36,0,0.77
19143,XS2376095068,2022-01-05,40.99,41.02,40.77,41.02,250,-0.81
19144,XS2376095068,2022-01-06,38.15,37.85,37.85,38.15,7,-7.73
19145,XS2376095068,2022-01-07,36.86,36.71,36.71,37.41,2541,-3.02
